[View in Colaboratory](https://colab.research.google.com/github/Sirius79/OpenAI/blob/master/openai_jokes_language_model.ipynb)

In [0]:
!wget https://raw.githubusercontent.com/taivop/joke-dataset/master/reddit_jokes.json
!wget https://raw.githubusercontent.com/taivop/joke-dataset/master/stupidstuff.json
!wget https://raw.githubusercontent.com/taivop/joke-dataset/master/wocka.json

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision
!pip install torchtext

In [41]:
import pandas as pd
import numpy as np
from numpy import interp
import json
import torch
import torchtext.vocab as vocab

In [29]:
r_jokes = pd.read_json('reddit_jokes.json')
s_jokes = pd.read_json('stupidstuff.json')
w_jokes = pd.read_json('wocka.json')

In [30]:
r_jokes.head()

,body,id,score,title
0,"Now I have to say ""Leroy can you please paint ...",5tz52q,1,I hate how you cant even say black paint anymore
1,Pizza doesn't scream when you put it in the ov...,5tz4dd,0,What's the difference between a Jew in Nazi Ge...
2,...and being there really helped me learn abou...,5tz319,0,I recently went to America....
3,A Sunday school teacher is concerned that his ...,5tz2wj,1,"Brian raises his hand and says, “He’s in Heaven.”"
4,He got caught trying to sell the two books to ...,5tz1pc,0,You hear about the University book store worke...


In [31]:
s_jokes.head()

,body,category,id,rating
0,A blackjack dealer and a player with a thirtee...,Children,1,2.63
1,"At a dinner party, several of the guests were ...",Blonde Jokes,2,2.57
2,One day this cop pulls over a blonde for speed...,Blonde Jokes,3,3.09
3,Three women are about to be executed for crime...,Blonde Jokes,4,4.10
4,A girl came skipping home FROM school one day....,Blonde Jokes,5,4.30


In [32]:
w_jokes.head()

,body,category,id,title
0,What do you call a cow with no legs?\r\n\r\nGr...,Animal,1,Cow With No Legs
1,What do you call a cow jumping over a barbed w...,Animal,2,Jumping Cow
2,What's black and white and red all over?\r\n\r...,Other / Misc,4,"Black, White and Red"
3,"So, this guy walks into a bar.\r\n\r\nAnd says...",Bar,5,Guy in a Bar
4,"If the opposite of pro is con, isn't the oppos...",One Liners,6,Progress


In [33]:
r_jokes.drop(columns=['id'], inplace=True)
r_jokes.head()

,body,score,title
0,"Now I have to say ""Leroy can you please paint ...",1,I hate how you cant even say black paint anymore
1,Pizza doesn't scream when you put it in the ov...,0,What's the difference between a Jew in Nazi Ge...
2,...and being there really helped me learn abou...,0,I recently went to America....
3,A Sunday school teacher is concerned that his ...,1,"Brian raises his hand and says, “He’s in Heaven.”"
4,He got caught trying to sell the two books to ...,0,You hear about the University book store worke...


In [34]:
r_jokes["joke"] = r_jokes["title"].map(str) + " \n " + r_jokes["body"].map(str)
r_jokes.head()

,body,score,title,joke
0,"Now I have to say ""Leroy can you please paint ...",1,I hate how you cant even say black paint anymore,I hate how you cant even say black paint anymo...
1,Pizza doesn't scream when you put it in the ov...,0,What's the difference between a Jew in Nazi Ge...,What's the difference between a Jew in Nazi Ge...
2,...and being there really helped me learn abou...,0,I recently went to America....,I recently went to America.... \n ...and being...
3,A Sunday school teacher is concerned that his ...,1,"Brian raises his hand and says, “He’s in Heaven.”","Brian raises his hand and says, “He’s in Heave..."
4,He got caught trying to sell the two books to ...,0,You hear about the University book store worke...,You hear about the University book store worke...


In [37]:
r_jokes["score"].min()
r_jokes["score"].max()

48526

In [38]:
r_jokes["joke"][4]

'You hear about the University book store worker who was charged for stealing $20,000 worth of books? \n He got caught trying to sell the two books to a freshman.'

In [39]:
r_jokes.drop(columns=['body','title'], inplace=True)
r_jokes.head()

,score,joke
0,1,I hate how you cant even say black paint anymo...
1,0,What's the difference between a Jew in Nazi Ge...
2,0,I recently went to America.... \n ...and being...
3,1,"Brian raises his hand and says, “He’s in Heave..."
4,0,You hear about the University book store worke...


In [42]:
val = interp(r_jokes.score.values, [0, 48526], [1, 10])
print(val)

[1.00018547 1.         1.         ... 1.00816057 1.01168446 1.        ]


In [43]:
r_jokes['score'] = val

In [44]:
r_jokes.score.max()

10.0

In [45]:
r_jokes['joke'][7]

'I walked into a PETA adoption center and the receptionist asked me what kind of dog I wanted \n Apparently "Whatever\'s low in cholesterol" was not the right answer.'

In [46]:
s_jokes.drop(columns=['category','id'], inplace=True)
s_jokes.head()

,body,rating
0,A blackjack dealer and a player with a thirtee...,2.63
1,"At a dinner party, several of the guests were ...",2.57
2,One day this cop pulls over a blonde for speed...,3.09
3,Three women are about to be executed for crime...,4.10
4,A girl came skipping home FROM school one day....,4.30


In [47]:
s_jokes = s_jokes.rename(columns={'body': 'joke', 'rating': 'score'})
s_jokes.head()

,joke,score
0,A blackjack dealer and a player with a thirtee...,2.63
1,"At a dinner party, several of the guests were ...",2.57
2,One day this cop pulls over a blonde for speed...,3.09
3,Three women are about to be executed for crime...,4.10
4,A girl came skipping home FROM school one day....,4.30


In [48]:
s_jokes = s_jokes[['score','joke']]

In [49]:
s_jokes.head()

,score,joke
0,2.63,A blackjack dealer and a player with a thirtee...
1,2.57,"At a dinner party, several of the guests were ..."
2,3.09,One day this cop pulls over a blonde for speed...
3,4.10,Three women are about to be executed for crime...
4,4.30,A girl came skipping home FROM school one day....


In [50]:
val = interp(s_jokes.score.values, [1, 5], [1, 10])
print(val)

[4.6675 4.5325 5.7025 ... 1.     7.75   5.5   ]


In [51]:
s_jokes['score'] = val

In [52]:
data = pd.concat([r_jokes, s_jokes])

In [53]:
data.shape

(198326, 2)

In [28]:
glove = vocab.GloVe(name='6B', dim=300)

print('Loaded {} words'.format(len(glove.itos)))

.vector_cache/glove.6B.zip: 862MB [01:03, 13.6MB/s]                           
 92%|█████████▏| 367614/400000 [00:50<00:04, 7315.11it/s]

100%|██████████| 400000/400000 [00:54<00:00, 7310.21it/s]


Loaded 400000 words


In [0]:
def get_word(word):
  return glove.vectors[glove.stoi[word]]

In [44]:
vector = get_word('microsoft')

print(vector)


-0.6931
-0.5403
 0.0296
-0.7620
-0.1420
 0.1930
-0.2919
-0.0456
 0.6191
-1.2621
-0.0891
 0.1066
 0.3033
-0.4280
-0.1952
 0.4072
-0.2164
-0.1868
 0.2145
 0.0453
 0.6970
 0.2106
 0.8118
-0.0398
-0.4938
-0.1101
 0.5436
-0.6324
-0.1401
-0.4686
-0.2391
 0.1562
 0.6937
-0.1282
-1.0577
-0.5012
-0.4048
-0.1725
-0.2489
-0.3538
 0.4675
 0.6646
-0.5726
 1.2395
 0.0736
 0.3270
-0.4157
-0.1743
-0.1028
-0.5482
 0.3181
 0.0311
 0.0401
 0.2772
 0.1662
 0.5678
 0.0749
-0.0183
-0.0250
-0.3743
 0.2376
 0.3067
 0.4385
-0.1985
 0.0096
 0.2931
-0.7051
 0.2105
 0.0695
-0.2831
-0.5261
-0.2524
 0.2945
 0.0516
 0.4239
 0.1266
 0.4750
-0.1004
 0.3106
 0.2485
-0.6093
-0.0804
-0.4771
-0.7622
 0.4163
-0.2847
-0.0279
-0.0146
 0.2592
-0.4684
-1.1202
 0.0264
-0.3795
 0.6349
 0.6731
 0.4025
-0.8242
-0.0679
 0.3856
-0.8131
-0.1730
-0.1248
-0.3540
-0.3534
 0.1079
-0.1073
-0.2050
-0.0404
-0.2779
-0.4257
-0.5099
-0.4758
-0.3119
 0.1480
-0.3407
-0.2170
-0.5224
-0.1685
-0.3167
 0.2309
-0.0396
-0.6129
-0.2166
-0.5474
 0.2081

In [39]:
data.head(10)  

,score,joke
0,0.0,I hate how you cant even say black paint anymo...
1,0.0,What's the difference between a Jew in Nazi Ge...
2,0.0,I recently went to America.... \n ...and being...
3,0.0,"Brian raises his hand and says, “He’s in Heave..."
4,0.0,You hear about the University book store worke...
5,0.0,Why is it unknown on how pterodactyls urinate ...
6,0.0,Why women need legs? \n To avoid leaving track...
7,0.0,I walked into a PETA adoption center and the r...
8,0.0,How did TV studios make words appear on screen...
9,0.0,How do you find Will Smith in the snow? \n You...


In [82]:
_jokes = {}
for _, row in data.iterrows():
  _jokes[row['joke']] = row['score']  

'''
for key, value in _jokes.items():
  print(key+" : "+ str(value))
'''

'\nfor key, value in _jokes.items():\n  print(key+" : "+ str(value))\n'